In [15]:
import pandas as pd
import numpy as np
import time
from datetime import date
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
t_order = pd.read_csv('D:\研究生\Chaos\JD\信贷需求/t_order.csv')
t_user  =pd.read_csv('D:\研究生\Chaos\JD\信贷需求/t_user.csv')



In [16]:
t1 = t_order
print(t1.head(5))

     uid    buy_time     price  qty  cate_id  discount
0  45370  2016-11-23  3.995009    1       22       0.0
1  66975  2016-11-23  3.269410    1       26       0.0
2  75358  2016-11-23  2.255235    1       14       0.0
3  40597  2016-11-23  1.635284    1       20       0.0
4  83886  2016-11-23  1.920573    2       22       0.0


In [17]:
def original(x):
    result = 5**(x)-1
    return result
             
t1['price'] = t1.price.apply(original)
t1['discount'] =t1.discount.apply(original) 
t1['consume'] = t1.price*t1.qty-t1.discount
t1['price_sum'] = t1.price*t1.qty
print(t1.head(5))

     uid    buy_time   price  qty  cate_id  discount  consume  price_sum
0  45370  2016-11-23  619.00    1       22       0.0   619.00     619.00
1  66975  2016-11-23  191.85    1       26       0.0   191.85     191.85
2  75358  2016-11-23   36.70    1       14       0.0    36.70      36.70
3  40597  2016-11-23   12.90    1       20       0.0    12.90      12.90
4  83886  2016-11-23   21.00    2       22       0.0    42.00      42.00


In [23]:
import time
def get_month(x):
    timeArray = time.strptime(x, "%Y-%m-%d")
    return timeArray.tm_mon
t1['buy_month'] = t1.buy_time.apply(get_month)

In [20]:
print(t1.head(5))

     uid    buy_time   price  qty  cate_id  discount  consume  price_sum  \
0  45370  2016-11-23  619.00    1       22       0.0   619.00     619.00   
1  66975  2016-11-23  191.85    1       26       0.0   191.85     191.85   
2  75358  2016-11-23   36.70    1       14       0.0    36.70      36.70   
3  40597  2016-11-23   12.90    1       20       0.0    12.90      12.90   
4  83886  2016-11-23   21.00    2       22       0.0    42.00      42.00   

  buy_month  
0        11  
1        11  
2        11  
3        11  
4        11  


In [43]:
#consume_mean指平均每次下单的消费额，4month_mean指4个月份平均每月的消费额，consume_8指8月总的消费额
t2 = t1[['uid','consume','buy_month']]
t2 = t2[(t2.consume>0)]
# t_all_mean = t2[['uid','consume']]
# t_all_mean = t2.groupby('uid')['consume'].mean().reset_index()
t2 = t2.groupby(['uid','buy_month']).sum().reset_index()
# t_4month_mean =  t2[['uid','consume']]
# t_4month_mean = t2.groupby('uid').sum().reset_index()
# t_4month_mean['4month_mean'] = t_4month_mean.consume/4
t_8 = t2[t2.buy_month==8]
t_9 = t2[t2.buy_month==9]
t_10 = t2[t2.buy_month==10]
t_11 = t2[t2.buy_month==11]
# t_4month_mean = t_4month_mean[['uid','4month_mean']]
# t_all_mean.columns=['uid','consume_mean']
t_8.columns=['uid','buy_month','consume_8']
t_9.columns=['uid','buy_month','consume_9']
t_10.columns=['uid','buy_month','consume_10']
t_11.columns=['uid','buy_month','consume_11']


In [44]:
consume = t_user[['uid']].drop_duplicates(['uid']).reset_index()
# consume = pd.merge(consume,t_all_mean, on=['uid'], how='left')
# consume = pd.merge(consume,t_4month_mean, on=['uid'], how='left')
consume = pd.merge(consume,t_8, on=['uid'], how='left')
consume = pd.merge(consume,t_9, on=['uid'], how='left')
consume = pd.merge(consume,t_10, on=['uid'], how='left')
consume = pd.merge(consume,t_11, on=['uid'], how='left')
consume = consume[['uid','consume_8','consume_9','consume_10','consume_11']]

In [45]:
#price
t2 = t1[['uid','price_sum','buy_month']]
# t_all_mean = t2[['uid','price_sum']]
# t_all_mean = t2.groupby('uid')['price_sum'].mean().reset_index()
t2 = t2.groupby(['uid','buy_month']).sum().reset_index()
# t_4month_mean =  t2[['uid','price_sum']]
# t_4month_mean = t2.groupby('uid').sum().reset_index()
# t_4month_mean['price_sum_4month_mean'] = t_4month_mean.price_sum/4
t_8 = t2[t2.buy_month==8]
t_9 = t2[t2.buy_month==9]
t_10 = t2[t2.buy_month==10]
t_11 = t2[t2.buy_month==11]
# t_4month_mean = t_4month_mean[['uid','price_sum_4month_mean']]
# t_all_mean.columns=['uid','price_sum_all_mean']
t_8.columns=['uid','buy_month','price_sum_8']
t_9.columns=['uid','buy_month','price_sum_9']
t_10.columns=['uid','buy_month','price_sum_10']
t_11.columns=['uid','buy_month','price_sum_11']

# consume2 = pd.merge(consume,t_all_mean, on=['uid'], how='left')
# consume2 = pd.merge(consume2,t_4month_mean, on=['uid'], how='left')
consume2 = pd.merge(consume2,t_8, on=['uid'], how='left')
consume2 = pd.merge(consume2,t_9, on=['uid'], how='left')
consume2 = pd.merge(consume2,t_10, on=['uid'], how='left')
consume2 = pd.merge(consume2,t_11, on=['uid'], how='left')

consume2 = consume2.fillna(0)
consume_11 = consume2[['uid','consume_8','consume_9','consume_10','price_sum_8','price_sum_9','price_sum_10']]
consume_11.columns = ['uid','consume_3','consume_2','consume_1','price_sum_3','price_sum_2','price_sum_1']

consume_12 = consume2[['uid','consume_9','consume_10','consume_11','price_sum_9','price_sum_10','price_sum_11']]
consume_12.columns = ['uid','consume_3','consume_2','consume_1','price_sum_3','price_sum_2','price_sum_1']

consume_11.to_csv('D:/研究生/Chaos/JD/信贷需求/feature11/consume11.csv',index=None) 
consume_12.to_csv('D:/研究生/Chaos/JD/信贷需求/feature12/consume12.csv',index=None) 